# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [21]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from datetime import datetime

In [22]:
#Set up connection to the database and define conn & cur
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [23]:
#Write a function to retrieve all root, directories, files available on a filepath provided
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [24]:
#Use the get_files function provided above and retrieved all song JSON files in data/song_data folder
song_files = get_files('data/song_data')

In [25]:
#Pick up the first JSON file from the list
filepath = song_files[0]

In [26]:
#Read JSON file through PANDAS function read_json
data = pd.read_json(filepath,lines=True)
print(data.head())

            artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0  


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [27]:
#Convert data to a Data Frame for easy execution & print values
df = pd.DataFrame(data)

#Select specific values from the dataframe and store in song_data
song_data = df[['song_id','title','artist_id','year','duration']]

#Convert song_data to a list
song_data = song_data.values.tolist()

song_data =song_data[0]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [28]:
cur.execute(song_table_insert,song_data)
conn.commit()

IntegrityError: duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOMZWCG12A8C13C480) already exists.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
#Select specific values from the dataframe and store in song_data
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]

#Convert song_data to a list
artist_data = artist_data.values.tolist()

artist_data = artist_data[0]    

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert,artist_data)

conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
#Get all the files from the log folder
log_files = get_files('data/log_data/2018/11')

In [ ]:
#Pick up the first file in the log folder
filepath = log_files[0]

In [ ]:
#Using PANDA functions, read through the contents of the first file and display that
df_log = pd.read_json(filepath, lines=True)
print(df_log)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
#Filter records by NextSong action & store it in a variable df_filter
df_filter = df_log[(df_log['page'] == 'NextSong')]

In [ ]:
#Convert data in ts column to date and time. Create another column timestamp and add that to the dataframe.    
df_filter.loc[:,'ts'] = (pd.to_datetime(df_filter['ts'], unit='ms'))

In [31]:
#Extract individual row content from Data Frame and then append it a list
time_data = [ df_filter["ts"].dt.time, df_filter["ts"].dt.hour, df_filter["ts"].dt.day, df_filter["ts"].dt.weekofyear, df_filter["ts"].dt.month, df_filter["ts"].dt.year, df_filter["ts"].dt.weekday]

#Specify labels for these columns and set to column labels
column_labels = ['start_time','hr','dd','WoY','mm','yy','weekday']

#Marry up the above data to create a dictionary using ZIP
time_data_dict=dict(zip(column_labels,time_data))

#Convert data dictionary to a dataframe
time_df=pd.DataFrame.from_dict(time_data_dict)
print(time_df.head())

        start_time  hr  dd  WoY  mm    yy  weekday
0  00:22:07.796000   0  30   48  11  2018        4
1  01:08:41.796000   1  30   48  11  2018        4
2  01:12:48.796000   1  30   48  11  2018        4
3  01:17:05.796000   1  30   48  11  2018        4
4  01:20:56.796000   1  30   48  11  2018        4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
#Insert dataframe data using function ITERTUPLES() in a for loop
try:
     for row in time_df.itertuples():
         cur.execute(time_table_insert,\
                      (row.start_time,
                       row.hr,
                       row.dd,
                       row.WoY,
                       row.mm,
                       row.yy,
                       row.weekday)
                   ) 

except psycopg2.Error as e: 
    print("Error: INsert error")
    print (e)


conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [32]:
#df_log contains user_id, f_name,l_name,gender,level. We need to transfer those values to the user data frame
user_df = df_log[['userId','firstName','lastName','gender','level']].copy()
user_df=user_df.dropna()
user_df=user_df.drop_duplicates()
print(user_df.head())

   userId firstName lastName gender level
0      91    Jayden     Bell      M  free
1      73     Jacob    Klein      M  paid
8       4    Alivia  Terrell      F  free
22     86     Aiden     Hess      M  free
28     24     Layla  Griffin      F  paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for row in user_df.itertuples():
    cur.execute(users_table_insert,\
                 (row.userId,
                  row.firstName,
                  row.lastName,
                  row.gender,
                  row.level)
               )       
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df_log.iterrows():
#get songid and artistid from song and artist tables
    cur.execute(song_select,(row.song,row.artist,row.length))
    results = cur.fetchone() 

    if results:
        songid,artistid = results
    else:
        songid, artistid = None, None 
   
    songplay_data = (index,row.ts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)      
    cur.execute(songplay_table_insert,songplay_data)            
  
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.